In [6]:
import requests
from datetime import datetime
import json
import pytz

LONG, LAT = (39.5789544215873, -74.22461197107408)


def get_daily_wind_data(
    latitude: float = LAT,
    longitude: float = LONG,
    times_to_get=("08:00", "12:00", "15:00", "18:00"),
):
    base_url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "wind_speed_10m,wind_direction_10m",
        # "daily": "wind_speed_10m_max,wind_direction_10m_dominant",
        "timezone": "America/New_York",
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    print(json.dumps(data, indent=4))

    selected = []

    for t, speed, direction in zip(
        data["hourly"]["time"],
        data["hourly"]["wind_speed_10m"],
        data["hourly"]["wind_direction_10m"],
    ):
        dt = datetime.fromisoformat(t)
        if dt.strftime("%H:%M") in times_to_get and dt.date() == datetime.now().date():
            selected.append(
                {
                    "time": dt.strftime("%-I %p"),  # e.g., "8 AM"
                    "speed_kmh": speed,
                    "direction_deg": direction,
                }
            )

    return selected


#


def get_wind_data(latitude=LAT, longitude=LONG, hours=24, days=3):
    # url = (
    #     "https://marine-api.open-meteo.com/v1/marine?"
    #     "latitude=39.565&longitude=-74.240&hourly=wind_speed_10m,wind_direction_10m&"
    #     "daily=wind_speed_10m_max,wind_direction_10m_dominant&timezone=auto"
    # )
    base_url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "wind_speed_10m,wind_direction_10m",
        # "daily": "wind_speed_10m_max,wind_direction_10m_dominant",
        "timezone": "America/New_York",
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    print(json.dumps(data, indent=4))

    # Convert strings to datetime
    hourly_times = [datetime.fromisoformat(t) for t in data["hourly"]["time"]]

    # Match the nearest time to now
    now = datetime.now().replace(minute=0, second=0, microsecond=0)
    try:
        hour_idx = hourly_times.index(now)
    except ValueError:
        raise RuntimeError(f"Current hour {now.isoformat()} not found in hourly times")

    current_wind_speed = data["hourly"]["wind_speed_10m"][hour_idx]
    current_wind_dir = data["hourly"]["wind_direction_10m"][hour_idx]

    # Get current hour index
    current_hour = datetime.now().strftime("%Y-%m-%dT%H:00")
    hour_idx = data["hourly"]["time"].index(current_hour)

    current_wind_speed = data["hourly"]["wind_speed_10m"][hour_idx]
    current_wind_dir = data["hourly"]["wind_direction_10m"][hour_idx]

    # Next 24 hours
    next_24 = {
        "wind_speeds": data["hourly"]["wind_speed_10m"][hour_idx : hour_idx + 24],
        "wind_dirs": data["hourly"]["wind_direction_10m"][hour_idx : hour_idx + 24],
        "times": data["hourly"]["time"][hour_idx : hour_idx + 24],
    }

    # # Next 3 days
    # next_3_days = list(
    #     zip(
    #         data["daily"]["time"][:3],
    #         data["daily"]["wind_speed_10m_max"][:3],
    #         data["daily"]["wind_direction_10m_dominant"][:3],
    #     )
    # )

    return {
        "current": {"speed": current_wind_speed, "dir": current_wind_dir},
        "next_24": next_24,
        # "next_3_days": next_3_days,
    }


# get_wind_data()
wind_data = get_daily_wind_data()

{
    "latitude": -74.25,
    "longitude": 39.625,
    "generationtime_ms": 2.387881278991699,
    "utc_offset_seconds": -14400,
    "timezone": "America/New_York",
    "timezone_abbreviation": "GMT-4",
    "elevation": 3375.0,
    "hourly_units": {
        "time": "iso8601",
        "wind_speed_10m": "km/h",
        "wind_direction_10m": "\u00b0"
    },
    "hourly": {
        "time": [
            "2025-07-04T00:00",
            "2025-07-04T01:00",
            "2025-07-04T02:00",
            "2025-07-04T03:00",
            "2025-07-04T04:00",
            "2025-07-04T05:00",
            "2025-07-04T06:00",
            "2025-07-04T07:00",
            "2025-07-04T08:00",
            "2025-07-04T09:00",
            "2025-07-04T10:00",
            "2025-07-04T11:00",
            "2025-07-04T12:00",
            "2025-07-04T13:00",
            "2025-07-04T14:00",
            "2025-07-04T15:00",
            "2025-07-04T16:00",
            "2025-07-04T17:00",
            "2025-07-04T18:00",
 

In [ ]:
def kmh_to_mph(kmh: float) -> float:
    return round(kmh * 0.621371, 1)


def deg_to_16_point_direction(deg):
    """
    Convert degrees to compass rose directions
    """
    directions = [
        "N",
        "NNE",
        "NE",
        "ENE",
        "E",
        "ESE",
        "SE",
        "SSE",
        "S",
        "SSW",
        "SW",
        "WSW",
        "W",
        "WNW",
        "NW",
        "NNW",
    ]
    ix = round(deg / 22.5) % 16
    return directions[ix]


def classify_wind_relative_to_beach(wind_deg, beach_facing_deg=140):
    # Normalize the angle difference between 0–180
    diff = abs(wind_deg - beach_facing_deg) % 360
    if diff > 180:
        diff = 360 - diff

    if diff <= 22.5:
        return "Onshore"
    elif diff <= 67.5:
        return "Onshore/Cross-shore"
    elif diff <= 112.5:
        return "Cross-shore"
    elif diff <= 157.5:
        return "Cross-shore/Offshore"
    else:
        return "Offshore"


for entry in wind_data:
    deg = entry["direction_deg"]
    entry["speed_mph"] = kmh_to_mph(entry["speed_kmh"])
    entry["direction"] = deg_to_16_point_direction(deg)
    entry["wind_type"] = classify_wind_relative_to_beach(deg)


wind_data

[{'time': '8 AM',
  'speed_kmh': 18.6,
  'direction_deg': 137,
  'speed_mph': 11.6,
  'direction': 'SE',
  'wind_type': 'Onshore'},
 {'time': '12 PM',
  'speed_kmh': 17.3,
  'direction_deg': 138,
  'speed_mph': 10.7,
  'direction': 'SE',
  'wind_type': 'Onshore'},
 {'time': '3 PM',
  'speed_kmh': 17.3,
  'direction_deg': 138,
  'speed_mph': 10.7,
  'direction': 'SE',
  'wind_type': 'Onshore'},
 {'time': '6 PM',
  'speed_kmh': 15.5,
  'direction_deg': 138,
  'speed_mph': 9.6,
  'direction': 'SE',
  'wind_type': 'Onshore'}]

In [14]:
lines = []
lines.append("**Wind Forecast for Today** 🌬️\n")
lines.append(
    "Here’s the wind forecast near your beach (facing 140°) for key times today:\n"
)

for entry in wind_data:
    line = f"- **{entry['time']}** – {entry['speed_mph']} mph ({entry['speed_kmh']} km/h) from **{entry['direction']} ({entry['direction_deg']}°)** → **{entry['wind_type']}**"
    lines.append(line)

lines.append(
    "\nAll winds are **onshore**, which may create choppier ocean conditions and less ideal surf."
)

email_body = "\n".join(lines)

print(email_body)

**Wind Forecast for Today** 🌬️

Here’s the wind forecast near your beach (facing 140°) for key times today:

- **8 AM** – 11.6 mph (18.6 km/h) from **SE (137°)** → **Onshore**
- **12 PM** – 10.7 mph (17.3 km/h) from **SE (138°)** → **Onshore**
- **3 PM** – 10.7 mph (17.3 km/h) from **SE (138°)** → **Onshore**
- **6 PM** – 9.6 mph (15.5 km/h) from **SE (138°)** → **Onshore**

All winds are **onshore**, which may create choppier ocean conditions and less ideal surf.


In [10]:
lines = []
lines.append("**Wind Forecast for Today** 🌬️\n")
lines.append(
    "Here’s the wind forecast near your beach (facing 140°) for key times today:\n"
)

for entry in wind_data:
    line = f"- **{entry['time']}** – {entry['speed_mph']} mph ({entry['speed_kmh']} km/h) from **{entry['direction']} ({entry['direction_deg']}°)** → **{entry['wind_type']}**"
    lines.append(line)

In [13]:
print("\n".join(lines))

**Wind Forecast for Today** 🌬️

Here’s the wind forecast near your beach (facing 140°) for key times today:

- **8 AM** – 11.6 mph (18.6 km/h) from **SE (137°)** → **Onshore**
- **12 PM** – 10.7 mph (17.3 km/h) from **SE (138°)** → **Onshore**
- **3 PM** – 10.7 mph (17.3 km/h) from **SE (138°)** → **Onshore**
- **6 PM** – 9.6 mph (15.5 km/h) from **SE (138°)** → **Onshore**
